UsingIP                    2  
LongURL                    3  
ShortURL                   2  
Symbol@                    2  
Redirecting//              2  
PrefixSuffix-              2  
SubDomains                 3  
HTTPS                      3  
DomainRegLen               2  
Favicon                    2  
NonStdPort                 2  
HTTPSDomainURL             2  
RequestURL                 2  
AnchorURL                  3  
LinksInScriptTags          3  
ServerFormHandler          3  
InfoEmail                  2  
AbnormalURL                2  
WebsiteForwarding          2  
StatusBarCust              2  
DisableRightClick          2  
UsingPopupWindow           2  
IframeRedirection          2  
AgeofDomain                2  
DNSRecording               2  
WebsiteTraffic             3  
PageRank                   2  
GoogleIndex                2  
LinksPointingToPage        3  
StatsReport                2  
class                      2  

In [4]:
import kagglehub
import pandas as pd
import os
# Download latest version
path = kagglehub.dataset_download("ndarvind/phiusiil-phishing-url-dataset")

print("Path to dataset files:", path)
df = pd.read_csv(os.path.join(path, [filename for filename in os.listdir(path) if filename.endswith('.csv')][0]))

Path to dataset files: C:\Users\rrpra\.cache\kagglehub\datasets\ndarvind\phiusiil-phishing-url-dataset\versions\1


In [13]:
df.info()
for i in df.columns:
    print(i,df[i].value_counts().shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235795 entries, 0 to 235794
Data columns (total 55 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   URL                         235795 non-null  object 
 1   URLLength                   235795 non-null  int64  
 2   Domain                      235795 non-null  object 
 3   DomainLength                235795 non-null  int64  
 4   IsDomainIP                  235795 non-null  int64  
 5   TLD                         235795 non-null  object 
 6   URLSimilarityIndex          235795 non-null  float64
 7   CharContinuationRate        235795 non-null  float64
 8   TLDLegitimateProb           235795 non-null  float64
 9   URLCharProb                 235795 non-null  float64
 10  TLDLength                   235795 non-null  int64  
 11  NoOfSubDomain               235795 non-null  int64  
 12  HasObfuscation              235795 non-null  int64  
 13  NoOfObfuscated